In [ ]:
import sidewalkify as swky
import osmnx as ox
import numpy as np
import pandas as pd
import networkx as nx
import shapely as shapely
import geopandas as gpd


In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

## Get the initial road map
Here's what osmnx thinks of as 'walking':

In [ ]:
ox.downloader._get_osm_filter("walk")

In [ ]:
brighton_streets = gpd.read_file("brighton_streets.shp")

In [ ]:
brighton_streets

In [ ]:
brighton_streets.plot()

In [ ]:
print(brighton_streets.crs)
EPSG_4326 = brighton_streets.crs

In [ ]:
def multis_to_lines(ls_or_mls):
    if type(ls_or_mls) == shapely.geometry.LineString:
        return [ls_or_mls]
    else:
        return list(ls_or_mls)
    
df_temp = pd.DataFrame(brighton_streets)
df_temp['geometry'] = df_temp['geometry'].map(multis_to_lines)
df_temp = df_temp.explode('geometry')
brighton_streets = gpd.GeoDataFrame(df_temp, geometry='geometry')
brighton_streets.crs = EPSG_4326
brighton_streets

In [ ]:
EPSG_26919 = "EPSG:26919"
brighton_streets = brighton_streets.to_crs(EPSG_26919)

In [ ]:
sidewalk_paths = swky.graph.graph_workflow(brighton_streets)

In [ ]:
brighton_sidewalks = swky.draw.draw_sidewalks(sidewalk_paths)
brighton_sidewalks.crs=EPSG_26919

In [ ]:
brighton_sidewalks.to_file("brighton_sidewalks")
brighton_streets.to_file("brighton_streets_26919")

In [ ]:
import math
def linestring_start(linestring):
    (l1,l2) = list(linestring.coords)[0]
    return (l1,l2)

def linestring_end(linestring):
    (l1,l2) = list(linestring.coords)[1]
    return (l1,l2)

def linestring_heading(linestring):
    # 0 is true north, 90 is east
    # so heading = 90 - usual_angle (in standard form)
    (l1,l2) = linestring_start(linestring)
    (m1,m2) = linestring_end(linestring)
    angle_deg = math.atan2(m2-l2, m1-l1) * 360 / (2 * math.pi)
    heading_deg = int((90 - angle_deg) % 360)
    return(heading_deg)

# probably need to rework this for GSV, ok
brighton_streets['heading'] = brighton_streets.geometry.map(linestring_heading)


Let's segment.

In [ ]:
brighton_streets

In [ ]:
brighton_streets = brighton_streets.to_crs(EPSG_4326)
brighton_streets['length'] = brighton_streets.to_crs("EPSG:26919").length
brighton_streets['ratio'] = brighton_streets.length/brighton_streets['length']
brighton_streets['ratio'].describe()

0.000011 seems reasonable

In [ ]:
import math
CUT_LENGTH_METERS = 8
CUT_LENGTH = 0.000011 * CUT_LENGTH_METERS
sum((brighton_streets.length/CUT_LENGTH).map(math.ceil))

In [ ]:
brighton_streets = brighton_streets.drop(columns=['ratio','highway','key'])

In [ ]:
def safe_linemerge(line):
    if type(line) == MultiLineString:
        return linemerge(line)
    else:
        return line

brighton_streets.geometry = brighton_streets.geometry.map(lambda x : safe_linemerge)

In [ ]:
# Thank you Sean Gilles https://gist.github.com/sgillies/465156#file_cut.py
from shapely.geometry import LineString, Point

def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]
        

# here's me
def recursive_cut(line, distance):
    if [line] == cut(line, distance):
        return [line]
    else:
        segment, rest = cut(line, distance)
        return [segment] + recursive_cut(rest, distance)

In [ ]:
# now we recursive_cut streets
brighton_streets_segmented = brighton_streets.copy()
brighton_streets_segmented['segments'] = brighton_streets.geometry.map(lambda x : recursive_cut(x,CUT_LENGTH))

df_temp = pd.DataFrame(brighton_streets_segmented)
df_temp = df_temp.explode('segments').drop(columns=['geometry'])
brighton_streets_segmented = gpd.GeoDataFrame(df_temp, geometry = 'segments')
brighton_streets_segmented.crs = EPSG_4326

In [ ]:
brighton_streets_segmented.length

In [ ]:
# get midpoints to figure out where to take photo
def midpoint(linestring):
    (x1,y1) = linestring_start(linestring)
    (x2,y2) = linestring_end(linestring)
    x_mid = (x2 + x1)/2
    y_mid = (y2 + y1)/2
    return Point(x_mid, y_mid)

brighton_streets_segmented['midpoints'] = brighton_streets_segmented.geometry.map(midpoint)

In [ ]:
list(brighton_streets_segmented.head(1)['segments'].values[0].coords)

Plan:
* feed each row to selenium
* * two pics, left and right
* * cut off for 'short' ones -- can add 'length_4269' to see where
* project little lines out and clip/delete bad sidewalks

In [ ]:
brighton_crosswalks = gpd.read_file("brighton_crosswalks/brighton_crosswalks.shp")

In [ ]:
brighton_crosswalks

In [ ]:
!which python